## Queries no Python com a biblioteca SQLite3

### Bibliotecas utilizadas

In [1]:
import sqlite3

### Conexão com a base de dados

In [2]:
conn = sqlite3.connect('../data/database.db')
c = conn.cursor()

### Consultas

#### Pergunta 1
Qual a quantidade de produtos únicos que foram vendidos?

In [3]:
c.execute('''  
SELECT
    COUNT(DISTINCT idProduto) AS qtProdutosUnicosVendidos

FROM vendas
          ''')

for row in c.fetchall():
    print (row)

(73,)


#### Pergunta 2
Quais os três produtos mais vendidos e as quantidades destes?

In [4]:
c.execute('''  
SELECT
    T1.idProduto,
    T2.descNomeProduto,
    T2.descTamanhoProduto,
    SUM(T1.qtProduto) AS somaQtProduto

FROM vendas AS T1

LEFT JOIN produtos AS T2
ON T1.idProduto = T2.idProduto

GROUP BY T1.idProduto

ORDER BY somaQtProduto DESC

LIMIT 3
          ''')

for row in c.fetchall():
    print (row)

(63, 'Casaco', 'G', 1020)
(60, 'Camisa', 'G', 1018)
(9, 'Blusa', 'G', 998)


#### Pergunta 3
Quais as quantidades mensais vendidas do produto camisa de tamanho G no ano de 2019?

In [5]:
c.execute('''  
SELECT
    strftime('%m', T1.dtVenda) AS dtMesVenda,
    SUM(T1.qtProduto) AS somaQtProduto

FROM vendas AS T1

LEFT JOIN produtos AS T2
ON T1.idProduto = T2.idProduto

WHERE strftime('%Y', T1.dtVenda) = '2019'
AND T2.descNomeProduto = 'Camisa' --  Produto mais vendido?
AND T2.descTamanhoProduto = 'G'

GROUP BY dtMesVenda
          ''')

for row in c.fetchall():
    print (row)

('01', 30)
('02', 22)
('03', 27)
('04', 31)
('05', 35)
('06', 56)
('07', 38)
('08', 65)
('09', 25)
('10', 30)
('11', 26)
('12', 56)


#### Pergunta 4
Qual a média da receita líquida mensal de vendas no ano de 2019?

In [6]:
c.execute('''  
WITH receita_mes AS (

    SELECT
        strftime('%m', dtVenda) AS dtMesVenda,
        SUM((vlPrecoUnitario*qtProduto)) AS vlSomaReceitaLiquida

    FROM vendas

    WHERE strftime('%Y', dtVenda) = '2019'

    GROUP BY dtMesVenda
)

SELECT
    AVG(vlSomaReceitaLiquida) AS vlMediaReceitaLiquida

FROM receita_mes
          ''')

for row in c.fetchall():
    print (row)

(324240.933333332,)


#### Pergunta 5
Qual loja teve a maior receita líquida de vendas no ano de 2019?

In [7]:
c.execute('''  
SELECT
    T2.idLoja,
    T2.descCidadeLoja,
    SUM(T1.vlPrecoUnitario * T1.qtProduto) AS vlSomaReceitaLiquida

FROM vendas AS T1

LEFT JOIN lojas AS T2
ON T1.idLoja = T2.idLoja

WHERE strftime('%Y', T1.dtVenda) = '2019'

GROUP BY T1.idLoja

ORDER BY vlSomaReceitaLiquida DESC

LIMIT 3
          ''')

for row in c.fetchall():
    print (row)

(1, 'Florianópolis', 992059.6000000201)
(2, 'Rio de Janeiro', 797032.6000000119)
(3, 'Porto Alegre', 736108.8000000068)
